In [14]:
import os,json,random
from glob import glob

In [15]:
data_list = dict()
data_list['training'] = []

In [16]:
data_set = []

In [17]:
#brats
paths = glob('/data/users/bol/monai_playground/brats_data/BraTS*')
for path in paths:
    # print(path)
    for mod in ['flair','t1','t1ce','t2']:
        image_path = glob(path+f'/*{mod}.nii.gz')[0]
        image_path = image_path.replace('/data/users/bol/monai_playground/','')
        data_set.append({'image':image_path,})

In [18]:
#glis
paths = glob('/data/users/bol/monai_playground/Glis_nifti/GLI*')
for path in paths:
    for mod in ['T1','T2']:
        image_path = glob(path+f'/*{mod}.nii.gz')[0]
        image_path = image_path.replace('/data/users/bol/monai_playground/','')
        data_set.append({'image':image_path,})

In [19]:
#PDGM
paths = glob('/data/users/bol/monai_playground/PDGM_nifti/UCSF-PDGM*')
for path in paths:
    for mod in ['ADC','ASL','FA','MD','DWI','FLAIR','SWI','T1','T2','T1c']:
        image_path = glob(path+f'/*{mod}.nii.gz')[0]
        image_path = image_path.replace('/data/users/bol/monai_playground/','')
        data_set.append({'image':image_path,})

In [20]:
random.shuffle(data_set)

In [21]:
len(data_set)

10474

In [15]:
data_list['training'] = data_set[:7000]

In [16]:
data_list['testing'] = data_set[7000:]

In [17]:
with open('all_image.json','w') as file:
    json.dump(data_list, file, indent=4)

In [37]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from monai import data
from monai import transforms
from monai.apps import MedNISTDataset, DecathlonDataset
from monai.config import print_config
from monai.data import CacheDataset, DataLoader
from monai.utils import first, set_determinism
from torch import nn
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from accelerate import Accelerator
# from generative.losses import PatchAdversarialLoss, PerceptualLoss
# from generative.networks.nets import AutoencoderKL, DiffusionModelUNet, PatchDiscriminator
# from generative.networks.schedulers import DDPMScheduler



In [38]:
with open('/data/users/bol/monai_playground/all_image.json') as f:
    json_data = json.load(f)
train_files = json_data['training']
validation_files = json_data['testing']

In [39]:
set_determinism(42)

accelerator = Accelerator()

train_transforms = transforms.Compose(
    [
        transforms.LoadImaged(keys=["image"]),
        # transforms.EnsureChannelFirstd(keys=["image"]),
        transforms.EnsureChannelFirstd(keys=["image"], channel_dim="no_channel"),
        transforms.EnsureTyped(keys=["image"]),
        transforms.Orientationd(keys=["image"], axcodes="RAS"),
        # transforms.Spacingd(keys=["image"], pixdim=(2, 2, 2), mode=("bilinear")),
        # transforms.CenterSpatialCropd(keys=["image"], roi_size=(80, 88, 64)),
        transforms.Spacingd(keys=["image"], pixdim=(1, 1, 1), mode=("bilinear")),
        transforms.CenterSpatialCropd(keys=["image"], roi_size=(160, 160, 128)),
        # transforms.Spacingd(keys=["image"], pixdim=(1.5, 1.5, 1.5), mode=("bilinear")),
        # transforms.CenterSpatialCropd(keys=["image"], roi_size=(120, 120, 96)),
        transforms.ScaleIntensityRangePercentilesd(keys="image", lower=0, upper=99.5, b_min=0, b_max=1),
        transforms.RandAffined(
            keys=["image"],
            rotate_range=(-np.pi / 36, np.pi / 36),
            translate_range=(-1, 1),
            scale_range=(-0.05, 0.05),
            padding_mode="zeros",
            prob=0.5,
        ),
    ]
)
train_ds = data.Dataset(data=train_files, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=12, persistent_workers=True)

val_transforms = transforms.Compose(
    [
        transforms.LoadImaged(keys=["image"]),
        # transforms.EnsureChannelFirstd(keys=["image"]),
        transforms.EnsureChannelFirstd(keys=["image"], channel_dim="no_channel"),
        transforms.EnsureTyped(keys=["image"]),
        transforms.Orientationd(keys=["image"], axcodes="RAS"),
        # transforms.Spacingd(keys=["image"], pixdim=(2, 2, 2), mode=("bilinear")),
        # transforms.CenterSpatialCropd(keys=["image"], roi_size=(80, 80, 64)),
        transforms.Spacingd(keys=["image"], pixdim=(1, 1, 1), mode=("bilinear")),
        transforms.CenterSpatialCropd(keys=["image"], roi_size=(160, 160, 128)),
        # transforms.Spacingd(keys=["image"], pixdim=(1.5, 1.5, 1.5), mode=("bilinear")),
        # transforms.CenterSpatialCropd(keys=["image"], roi_size=(120, 120, 96)),
        transforms.ScaleIntensityRangePercentilesd(keys="image", lower=0, upper=99.5, b_min=0, b_max=1),
    ]
)
val_ds = data.Dataset(data=validation_files, transform=train_transforms)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=True, num_workers=4)


In [26]:
import nibabel as nib

In [32]:
for test_data in train_files:
    # print(data['image'])
    image_nib = nib.load(test_data['image'])
    if len(image_nib.shape) != 3:
        print(len(image_nib.shape))
    # break

In [43]:
for test_data in train_ds:
    if test_data['image'].shape != torch.Size([1, 160, 160, 128]):
        print(test_data['image'].shape)

In [1]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# 加载日志文件
event_accumulator = EventAccumulator("/data/users/bol/StableDiffusionExp/log/events.out.tfevents.1720476323.80cfae04e42b.867.0")
event_accumulator.Reload()  # 加载日志内容

# 获取所有标量的 tags
tags = event_accumulator.Tags()["scalars"]

# 遍历每个 tag 并打印对应的数据
for tag in tags:
    events = event_accumulator.Scalars(tag)
    print(f"Tag: {tag}")
    for event in events:
        print(f"Step: {event.step}, Value: {event.value}")


Tag: recons_loss
Step: 0, Value: 0.39317965507507324
Step: 0, Value: 0.477398157119751
Step: 0, Value: 0.47550323605537415
Step: 0, Value: 0.498798131942749
Step: 0, Value: 0.4922933578491211
Step: 0, Value: 0.4987505376338959
Step: 0, Value: 0.49082091450691223
Step: 0, Value: 0.4907820224761963
Step: 0, Value: 0.4955156445503235
Step: 0, Value: 0.48577818274497986
Step: 0, Value: 0.48852187395095825
Step: 0, Value: 0.4860585629940033
Step: 0, Value: 0.49025899171829224
Step: 0, Value: 0.4912388026714325
Step: 0, Value: 0.4925704300403595
Step: 0, Value: 0.4923005998134613
Step: 0, Value: 0.5014438033103943
Step: 0, Value: 0.5018182992935181
Step: 0, Value: 0.49595120549201965
Step: 0, Value: 0.4979179799556732
Step: 0, Value: 0.5014830231666565
Step: 0, Value: 0.5022581219673157
Step: 0, Value: 0.5019201636314392
Step: 0, Value: 0.500249981880188
Step: 0, Value: 0.5053717494010925
Step: 0, Value: 0.5077357888221741
Step: 0, Value: 0.5109089612960815
Step: 0, Value: 0.5096281170845032